In [2]:
import time
import selenium
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.chrome.service import Service
import pandas as pd


In [1]:
# CSV파일 읽기
file = 'data/db_entertain.csv'
df = pd.read_csv(file)
df

NameError: name 'pd' is not defined

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm

### 특정 장소에 해당하는 카페 정보 크롤링

In [1]:
name_list2=[]
for n in tqdm(df['장소']):
    service = Service(executable_path='chromedriver.exe') 
    options=webdriver.ChromeOptions()
    driver=webdriver.Chrome(service=service,options=options)
    driver.get("https://map.naver.com/v5/")
    driver.implicitly_wait(5) # 창 오픈까지 최대 10초를 기다려줌
    time.sleep(2)
    
    # 키워드 변경가능
    search = f"{n} 카페"
    
    # 검색창에 검색어 입력하기
    search_box = driver.find_element(By.CSS_SELECTOR,"div.input_box>input.input_search")
    search_box.send_keys(search)
    time.sleep(2)

    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)

    try:
        frame=driver.find_element(By.CSS_SELECTOR,"iframe#searchIframe")

        # print(frame)
        driver.switch_to.frame(frame)
        
        nav=driver.find_elements(By.CSS_SELECTOR,"div > span.place_bluelink.TYaxT")
        for i in nav:
            print(i.text)

        cnt=1
        for i in nav:
            if cnt==1:
                # 클릭
                i.click()
                time.sleep(2)
                driver.switch_to.default_content()
                #-----------------------------------------------------------------------
                # 두번째 프레임
                entframe=driver.find_element(By.CSS_SELECTOR,"iframe#entryIframe")
                print(entframe)
                driver.switch_to.frame(entframe)
                # 장소명, 별점, 주소 
                name=driver.find_element(By.CLASS_NAME,'Fc1rA').text
                try:
                    star=driver.find_element(By.CSS_SELECTOR,'span.PXMot.LXIwF > em').text
                except:
                    star='NaN'
                addr=driver.find_element(By.CLASS_NAME,'LDgIH').text
                name_list2.append([name,star,addr])
                print(name,star,addr)
                # 두번째 프레임 나오기
                driver.switch_to.default_content()
                #-----------------------------------------------------------------------
                cnt+=1
            else:
                driver.switch_to.frame(frame)
                # 클릭
                i.click()
                time.sleep(2)
                driver.switch_to.default_content()
                #-----------------------------------------------------------------------
                # 두번째 프레임
                entframe=driver.find_element(By.CSS_SELECTOR,"iframe#entryIframe")
                print(entframe)
                driver.switch_to.frame(entframe)
                # 장소명, 별점, 주소 
                name=driver.find_element(By.CLASS_NAME,'Fc1rA').text
                try:
                    star=driver.find_element(By.CSS_SELECTOR,'span.PXMot.LXIwF > em').text
                except:
                    star='NaN'
                addr=driver.find_element(By.CLASS_NAME,'LDgIH').text
                name_list2.append([name,star,addr])
                # driver.switch_to.default_content()
                print(name,star,addr)
        driver.quit()
    except:
        driver.quit()

NameError: name 'tqdm' is not defined

### 리스트에 받아온 데이터 갯수 정보

In [7]:
len(name_list2)

243

### 리스트의 내용 CSV파일로 저장

In [8]:
import csv

# CSV 파일 경로
csv_file_path = 'places_cafe1.csv'

# CSV 파일 열기 (newline=''는 빈 라인을 방지하기 위함)
with open(csv_file_path, 'w', newline='') as file:
    # CSV writer 생성
    writer = csv.writer(file)
    
    # 데이터를 CSV 파일에 쓰기
    for row in name_list2:
        writer.writerow(row)

print('CSV 파일이 생성되었습니다:', csv_file_path)

CSV 파일이 생성되었습니다: places_cafe1.csv


### 네이버지도 페이지별 정보 크롤링

In [46]:
service = Service(executable_path='chromedriver.exe')
options=webdriver.ChromeOptions()
driver=webdriver.Chrome(service=service,options=options)
driver.get("https://map.naver.com/p/search/%EB%8C%80%EA%B5%AC%EC%B9%B4%ED%8E%98?c=10.00,0,0,0,dh")
driver.implicitly_wait(10) # 창 오픈까지 최대 10초를 기다려줌
time.sleep(2)

#searchIframe

name_list=[]
# frame = driver.find_element(By.TAG_NAME, 'iframe')

# 첫번째 프레임
frame=driver.find_element(By.CSS_SELECTOR,"iframe#searchIframe")
# print(frame)
# driver.switch_to.frame(frame)
driver.switch_to.frame(frame)


#_pcmap_list_scroll_container > ul > li:nth-child(3) > div.CHC5F > a.tzwk0

# 페이지 넘기기
pnum=1
pagenumlist={'1'}

while str(pnum) in pagenumlist:
    pagenumlist=[]
    plist=driver.find_elements(By.CLASS_NAME,"mBN2s ")
    for page in plist:
        pagenumlist.append(page.text)

    # 스크롤 끝까지 내리기
    scroll = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
    driver.execute_script("arguments[0].scrollBy(0,2000)", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll)
    time.sleep(2)

    # 페이지 넘기기
    rpage=driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]')
    # 대구 놀거리 
    nav=driver.find_elements(By.CSS_SELECTOR,"div > span.place_bluelink.TYaxT")
    print(nav)


    cnt=1
    for i in nav:
        if cnt==1:
            # 클릭
            i.click()
            time.sleep(2)
            driver.switch_to.default_content()
            #-----------------------------------------------------------------------
            # 두번째 프레임
            entframe=driver.find_element(By.CSS_SELECTOR,"iframe#entryIframe")
            print(entframe)
            driver.switch_to.frame(entframe)
            # 장소명, 별점, 주소 
            name=driver.find_element(By.CLASS_NAME,'Fc1rA').text
            try:
                star=driver.find_element(By.CSS_SELECTOR,'span.PXMot.LXIwF > em').text
            except:
                star='NaN'
            addr=driver.find_element(By.CLASS_NAME,'LDgIH').text
            name_list.append([name,star,addr])
            print(name,star,addr)
            # 두번째 프레임 나오기
            driver.switch_to.default_content()
            #-----------------------------------------------------------------------
            cnt+=1
        else:
            driver.switch_to.frame(frame)
            # 클릭
            i.click()
            time.sleep(2)
            driver.switch_to.default_content()
            #-----------------------------------------------------------------------
            # 두번째 프레임
            entframe=driver.find_element(By.CSS_SELECTOR,"iframe#entryIframe")
            print(entframe)
            driver.switch_to.frame(entframe)
            # 장소명, 별점, 주소 
            name=driver.find_element(By.CLASS_NAME,'Fc1rA').text
            try:
                star=driver.find_element(By.CSS_SELECTOR,'span.PXMot.LXIwF > em').text
            except:
                star='NaN'
            addr=driver.find_element(By.CLASS_NAME,'LDgIH').text
            name_list.append([name,star,addr])
            print(name,star,addr)
            # 두번째 프레임 나오기
            driver.switch_to.default_content()
            #-----------------------------------------------------------------------
            cnt+=1
        # 페이지 넘기기  
    driver.switch_to.frame(frame)  
    rpage[0].click()

    # rpage=driver.find_elements(By.CSS_SELECTOR,"a:nth-child(7) > svg")
    # rpage[0].click()
    time.sleep(2)
    pnum+=1

[<selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e74701eaa", element="91E91DDFE8CEDC729E95A67F2B6FDC0E_element_146")>, <selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e74701eaa", element="91E91DDFE8CEDC729E95A67F2B6FDC0E_element_147")>, <selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e74701eaa", element="91E91DDFE8CEDC729E95A67F2B6FDC0E_element_148")>, <selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e74701eaa", element="91E91DDFE8CEDC729E95A67F2B6FDC0E_element_149")>, <selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e74701eaa", element="91E91DDFE8CEDC729E95A67F2B6FDC0E_element_150")>, <selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e74701eaa", element="91E91DDFE8CEDC729E95A67F2B6FDC0E_element_151")>, <selenium.webdriver.remote.webelement.WebElement (session="b666f143fa63ea2352403c3e7470